# Python with PostgreSQL & PostGIS

Note that a PostgreSQL/PostGIS installation and an import of OpenStreetMap data into a database is required for this exercise!

For a PostgreSQL/PostGIS installation on your local computer see: https://postgis.net/workshops/postgis-intro/installation.html.

On PCs with amd64 architechture (not arm64 like Apple M1/M2) you can use the file docker-compose.yml.

For the import of OpenStreetMap data, the CLI tool osm2pgsql can be used (Windows/macOS).

For examples of how to use osm2pgsql see: https://osm2pgsql.org/examples.

## Libraries and Settings

In [ ]:
# Libraries
import os
import folium
import psycopg2
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

# Set up the database connection
user = "postgres"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

## Show tables in the database

In [ ]:
# Create a connection
conn = psycopg2.connect(user=user,
                        password=password,
                        host=host,
                        port=port,
                        database=database)

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")

# Fetch all the rows
tables = cursor.fetchall()

try:
    # Print rows
    for table in tables:
        print(table)
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    # Close connection
    if conn:
        cursor.close()
        conn.close()

## Show columns and data types of selected table

In [ ]:
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Close the connection
conn.close()

# Print the DataFrame
df

## Query spatial data from PostgreSQL database (1st example)

In [ ]:
# Create a connection
conn = create_engine(db_connection_url)  

# Query the database    
sql = """SELECT
        p.osm_id,
        p."addr:street",
        p."addr:housenumber",
        p."addr:city",
        p."addr:postcode",
        p.building,
        st_transform(p.way, 4326) AS geom
        FROM
        public.planet_osm_polygon AS p
        WHERE
        p."addr:street" IS NOT NULL
        AND p."addr:city" = 'Zürich'
        AND p."addr:postcode" IN ('8001')"""

# Create a GeoDataFrame
gdf_01 = gpd.GeoDataFrame.from_postgis(sql, conn)
gdf_01

# Close the connection
conn.dispose()

## Show the map

In [ ]:
# Extract the x (longitude) and y (latitude) coordinates from each polygon
lon = gdf_01.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf_01.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map (use grayscale tiles for better contrast)
m = folium.Map(location=[lat, lon], 
               zoom_start=15, 
               tiles='CartoDB positron')

# Map settings
folium.Choropleth(
    geo_data=gdf_01,
    name='map',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query spatial data from PostgreSQL database (2nd example)

In [ ]:
# Create a connection
conn = create_engine(db_connection_url)

# Query roads   
sql_a = """--Select major roads
           SELECT
           ST_TRANSFORM(p.way, 4326) AS geom
           FROM public.planet_osm_roads AS p
           WHERE
           highway IN ('motorway', 'trunk', 'primary')"""

# Create a GeoDataFrame
gdf_02a = gpd.GeoDataFrame.from_postgis(sql_a, conn, geom_col='geom')
gdf_02a

# Create Bufferes
sql_b = """--Create buffers around major roads and combine these buffers to one single buffer
        SELECT 
        1 as group_id,
        ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 200)), 4326) AS combined_buffer_geom
        FROM public.planet_osm_roads AS p
        WHERE
        highway IN ('motorway', 'trunk', 'primary')"""

# Create a GeoDataFrame
gdf_02b = gpd.GeoDataFrame.from_postgis(sql_b, conn, geom_col='combined_buffer_geom')
gdf_02b

# Close the connection
conn.dispose()

## Show the map

In [ ]:
# Extract the x (longitude) and y (latitude) coordinates from each polygon
# lon = gdf_02.geometry.apply(lambda polygon: polygon.centroid.x).mean()
# lat = gdf_02.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Use city of Zürich as center
lon = 8.54104
lat = 47.37445

# Initialize the map (use grayscale tiles for better contrast)
m = folium.Map(location=[lat, lon], 
               zoom_start=14, 
               tiles='CartoDB positron')

# Add buffer to map
buffer_group = folium.FeatureGroup(name='Buffer')
folium.Choropleth(
    geo_data=gdf_02b,
    fill_color='greenyellow'
).add_to(buffer_group)
buffer_group.add_to(m)

# Add roads to map
roads_group = folium.FeatureGroup(name='Roads')
folium.GeoJson(
    gdf_02a,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3
    }
).add_to(roads_group)
roads_group.add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

## Query spatial data from PostgreSQL database (3rd example)

In [ ]:
# Create a connection
conn = create_engine(db_connection_url)  

# Define the query
sql = """WITH buffer AS (
        -- Create buffers around major roads and combine these buffers to one single buffer
        SELECT 
        1 as group_id,
        ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 200)), 4326) AS combined_buffer_geom
        FROM public.planet_osm_roads AS p
        WHERE
        -- Filter for major roads
        highway IN ('motorway', 'trunk', 'primary')
        )
        -- Select all buildings within the buffer
        SELECT
        p.osm_id,
        p."addr:street",
        p."addr:housenumber",
        p."addr:city",
        p."addr:postcode",
        p.building,
        ST_Transform(p.way, 4326) AS geom
        FROM
        public.planet_osm_polygon AS p, buffer
        WHERE
        p."addr:city" IN ('Zürich')
        -- Filter for buildings within the buffer
        AND ST_Contains(buffer.combined_buffer_geom, ST_Transform(p.way, 4326));
        """

# Create a GeoDataFrame
gdf_03 = gpd.GeoDataFrame.from_postgis(sql, conn, geom_col='geom')

# Close the connection
conn.dispose()

gdf_03.head()

## Show map

In [ ]:
# Extract the x (longitude) and y (latitude) coordinates from each polygon
# lon = gdf_03.geometry.apply(lambda polygon: polygon.centroid.x).mean()
# lat = gdf_03.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Use city of Zürich as center
lon = 8.54104
lat = 47.37445

# Initialize the map (use grayscale tiles for better contrast)
m = folium.Map(location=[lat, lon], 
               zoom_start=14, 
               tiles='CartoDB positron')

# Create feature groups
buffer_group = folium.FeatureGroup(name='Buffer')
roads_group = folium.FeatureGroup(name='Roads')
bldgs_group = folium.FeatureGroup(name='Buildings')

# Add buffer to map
folium.Choropleth(
    geo_data=gdf_02b,
    fill_color='greenyellow'
).add_to(buffer_group)

# Add roads to map
folium.GeoJson(
    gdf_02a,
    style_function=lambda feature: {
        'color': 'red',
        'weight': 3
    }
).add_to(roads_group)

# Add buildigs in buffer to map
folium.Choropleth(
    geo_data=gdf_03,
    name='map',
    fill_color='gray'
).add_to(bldgs_group)

# Add feature groups to map
m.add_child(buffer_group)
m.add_child(roads_group)
m.add_child(bldgs_group)

folium.LayerControl().add_to(m)

# Plot map
m

## Query spatial data from PostgreSQL database (4th example)

In [ ]:
# Create a connection
conn = create_engine(db_connection_url)  

# Query the database    
sql = """WITH area_calculation AS (
        SELECT 
                osm_id,
                name,
                ST_Area(ST_Transform(way, 32632)) / 1000000 AS area_km2,
                ST_Transform(way, 4326) AS geom
        FROM planet_osm_polygon
        WHERE 
                boundary = 'administrative' 
                AND admin_level = '8'
        )
        SELECT *
        FROM area_calculation
        WHERE area_km2 >= 50;"""

# Create a GeoDataFrame
gdf_04 = gpd.GeoDataFrame.from_postgis(sql, conn, geom_col='geom')

# Close the connection
conn.dispose()

# Show query results
gdf_04

## Show the map

In [ ]:
# Extract the x (longitude) and y (latitude) coordinates from each polygon
lon = gdf_04.geometry.apply(lambda polygon: polygon.centroid.x).mean()
lat = gdf_04.geometry.apply(lambda polygon: polygon.centroid.y).mean()

# Initialize the map (use grayscale tiles for better contrast)
m = folium.Map(location=[lat, lon], 
               zoom_start=8, 
               tiles='CartoDB positron')

# Map settings
folium.Choropleth(
    geo_data=gdf_04,
    name='map',
    fill_color='greenyellow'
).add_to(m)

folium.LayerControl().add_to(m)

# Plot map
m

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')